In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster
client = Client()
client

In [ ]:
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

In [ ]:
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/drone/radar_data/20230621-bench-prf-testing/20230621_163428"
#prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/drone/radar_data/20230621-bench-prf/20230622_104230"
prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/drone/radar_data/20230621-bench-prf/20230621_173826"

zero_sample_idx = 159

zarr_path = pr.save_radar_data_to_zarr(prefix, zarr_base_location="/home/thomas/Documents/StanfordGrad/RadioGlaciology/test_tmp_zarr_cache/", skip_if_cached=True)
# pr.save_radar_data_to_zarr(prefix, zarr_base_location="/home/thomas/Documents/StanfordGrad/RadioGlaciology/test_tmp_zarr_cache/", dryrun=True)

zarr_path

In [ ]:
raw = xr.open_zarr(zarr_path).radar_data

raw

In [ ]:
stacked = pr.stack(raw, 1000)

In [ ]:
chirp_ts, chirp = generate_chirp(stacked.config)
compressed = pr.pulse_compress(stacked, chirp, fs=stacked.config['GENERATE']['sample_rate'], zero_sample_idx=zero_sample_idx)

In [ ]:
sig_speed = scipy.constants.speed_of_light * (2/3)
plot_dask.plot_radargram(compressed, sig_speed=sig_speed, figsize=(10,6), ylims=(-100,2000), vmax=0, vmin=-70)
plt.show()

In [ ]:
plt.plot(compressed.fast_time * (sig_speed/2), 20*np.log10(np.abs(compressed[:,20].compute())))
plt.xlim(-200, 5000)
plt.xlabel("Distance to reflector [m]")
plt.grid()